In [ ]:
from aiida import load_profile,orm, engine
from aiida.common.datastructures import StashMode
from datetime import datetime
from dateutil.relativedelta import relativedelta
load_profile();
import ipywidgets as widgets
from IPython.display import display
from widgets import sens_query, inversion_params, import_nc,presettings
import aiidalab_widgets_base as awb
from settings import *


out = widgets.Output()

In [ ]:
def codes_list():
    return [
        awb.ComputationalResourcesWidget._full_code_label(c[0])
        for c in orm.QueryBuilder().append(orm.Code).all()
    ]
def default(name):
    list_codes = codes_list()
    return name if name in list_codes else list_codes[0]

def get_dict_from_filter():
    filter_dict  ={}
    for k,v in sq.site_filter.items():
        filter_={}
        for i in v.children[0].children[0].children:
            f = {}
            for j in range(4):
                f[i.children[j].description] = i.children[j].value
            filter_[i.children[0].value]=f
        filter_dict[k] = filter_
    return filter_dict
   

# Inversion

### Import data

In [ ]:
import_nc = import_nc.Import()
import_nc

### Step 1

In [ ]:
sq = sens_query.SearchSens()
sq

### Step 2

In [ ]:
inv = inversion_params.InversionParams()
inv


### Step 3

In [ ]:
codes = widgets.Dropdown(
            description="code",
            options=codes_list(),
            value = default('inversion@daint'),
            style = {"description_width": "initial"}
        )


selected_code = orm.load_code(codes.value)
username = selected_code.computer.get_configuration()['username']


stash_address = widgets.Text(
            description="Stash Address",
            value = '/store/empa/em05/'+username+'/aiida_stash',
            style = {"description_width": "initial"},
            layout=widgets.Layout(width="40%"),
        )


block = widgets.VBox(children = [codes,
                                stash_address,
                                 ])
block


In [ ]:
prepend_text_ = f'#SBATCH --constrain=mc\n'+\
                f'#SBATCH --account=em05\n'
              

In [ ]:
def prepare_inversion():

    if not inv.inv_name.value:
        return
    
    inverstion_parameters = inv.construct_dict()
    filter_dict = get_dict_from_filter()

    extras_dict = {k:{x.description:x.value for x in v} for k,v in sq.site_extras.items()}
    for k in extras_dict.keys():
        extras_dict[k]['bg.obs.fn'] = sq.available_obs_list[extras_dict[k]['bg.obs.fn']].attributes["remote_path"]

    for i in sq.list_info_obs:
        i.update(extras_dict[i['name']])
        i.update(filter_dict[i['name']])

    inverstion_parameters.update({'sites':{i['name']:i for i in sq.list_info_obs}})

    #other params
    inverstion_parameters.update({'para': sq.species.value,
                                'domain':sq.domain.value,
                                'transport.model':sq.model.value,
                                #'dtm.start':s_date,
                                #'dtm.end':e_date,
                                'res.dir':'./'})
    
    #test conversion
    inverstion_parameters = {
    key.replace(".", "_"): value for key, value in inverstion_parameters.items()
}
    remotes_dictionary = {k[:-3].replace('-','_'):v 
                         for k,v in sq.list_remotes.items()}
    
    #selected obs format change
    reformated_observations = {k.replace('-','_'):v 
                               for k,v in sq.selected_obs.items()}
    
    #builder starts
    builder = INVERSION_WORKLFOW.get_builder()
    builder.inversion_code = orm.load_code(codes.value)
    builder.sens = remotes_dictionary
    builder.observations =  reformated_observations
    #builder.metadata.options.custom_scheduler_commands = prepend_text_
    builder.inv_params = orm.Dict(inverstion_parameters)
    builder.date_range = orm.Str(sq.date_range.value)
    builder.chunk = orm.Str(inv.chunk.value)
    builder.chunk_w = orm.Str(inv.chunk_w.value)

    """builder.metadata.options.stash = {
        'source_list': ['*'],
        'target_base': stash_address.value,
        'stash_mode': StashMode.COPY.value,
    }"""

    return builder


In [ ]:
pre = presettings.Presettings(INVERSION,
                              parameters = {'params':inv})
pre

In [ ]:
btn_submit_ = awb.SubmitButtonWidget(
    INVERSION_WORKLFOW,
    inputs_generator = prepare_inversion,
    disable_after_submit = False,
    append_output = True,
)
btn_submit_.btn_submit.button_style="success"
display(btn_submit_, out)